In [148]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
%time food_data = pd.read_csv('./csv_data_mail.ru/cooking.txt.csv')

food_data.shape

CPU times: user 2.13 s, sys: 247 ms, total: 2.38 s
Wall time: 2.38 s


(257761, 7)

In [4]:
food_data.category_urlname.value_counts()

food_other         69713
sweets_baking      38530
drinks             30197
main_courses       28114
food_buy           21170
food_howto         14242
salads             13454
food_devices       11874
food_preserving     9762
soups               8710
celebration         7988
food_kid            4007
Name: category_urlname, dtype: int64

In [5]:
food_categories = food_data.category_urlname.unique()
food_categories

array(['sweets_baking', 'drinks', 'food_devices', 'main_courses',
       'food_other', 'celebration', 'food_howto', 'food_buy', 'salads',
       'food_preserving', 'soups', 'food_kid'], dtype=object)

In [7]:
%time all_data = pd.read_csv('merged.csv')

CPU times: user 42.4 s, sys: 3.61 s, total: 46 s
Wall time: 46.1 s


In [10]:
correct_data = all_data.dropna()

In [11]:
correct_data.shape

(6577340, 6)

In [17]:
correct_data.category_urlname.value_counts()

humor               891219
adult               444122
love_other          303557
psychology          297445
community           296229
newcomers           196660
other               191329
drama               155501
religion            146965
family_other        120362
politics            108853
relations           100037
food_other           90590
football             89217
internet             87950
home_fauna           87515
hardware             83785
pc_games             83566
loving               80813
music                79589
software             75008
wisdom               73103
illness              66352
pc_other             63283
auto_other           62293
chiromancy           60197
auto_tuning          58388
other_mysterious     55442
edu_homework         53486
magia                53145
                     ...  
law_social            2745
autosport             2578
edu_baby              2546
production            2499
spa                   2397
sunburn               2027
r

Bag of bigram+unigram vectorizer

In [21]:
vectorizer = CountVectorizer(ngram_range=(1,2))
vectorizer.fit(correct_data.question)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [118]:
save_object(vectorizer, 'vectorizer_bigram+unigram.pkl')

Bag of unigram vectorizer

In [115]:
%%time 
vectorizer_unigram = CountVectorizer(ngram_range=(1,1))
vectorizer_unigram.fit(correct_data.question)

CPU times: user 1min 44s, sys: 10 s, total: 1min 54s
Wall time: 2min 2s


In [117]:
save_object(vectorizer_unigram, 'vectorizer_unigram.pkl')

bag of bigram vectorizer

In [119]:
%%time 
vectorizer_bigram = CountVectorizer(ngram_range=(2,2))
vectorizer_bigram.fit(correct_data.question)

CPU times: user 4min 46s, sys: 13.6 s, total: 5min
Wall time: 5min 4s


In [120]:
save_object(vectorizer_bigram, 'vectorizer_bigram.pkl')

Simple Baseline on uni grams

In [82]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression

In [149]:
#def make_classification(vectorizer, X, y)

In [121]:
%time X, y = vectorizer_unigram.transform(correct_data.question), correct_data.category_urlname.apply(lambda x: 1 if x in food_categories else 0)

CPU times: user 2min 1s, sys: 6.21 s, total: 2min 8s
Wall time: 2min 12s


In [122]:
y.value_counts()

0    6231106
1     346234
Name: category_urlname, dtype: int64

Сбалансируем выборку

In [123]:
import numpy as np
n = y.value_counts()[1]
mask = np.hstack([np.random.choice(np.where(y == l)[0], n, replace=False)
                      for l in np.unique(y)])

X_balanced, y_balanced = X[mask], y[mask]

In [124]:
from sklearn.cross_validation import train_test_split

In [125]:
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.3)

In [126]:
a = list(map(lambda x : 1 if x else 0, y_train))
b = list(map(lambda x : 1 if x else 0, y_test))

In [144]:
clf =  LogisticRegression(C=0.1)
clf.fit(X_train, a)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [145]:
from sklearn.metrics import classification_report
print(classification_report(b,  clf.predict(X_test)))

             precision    recall  f1-score   support

          0       0.83      0.84      0.84    128070
          1       0.74      0.72      0.73     79671

avg / total       0.80      0.80      0.80    207741



In [146]:
clf =  BernoulliNB(.01)
clf.fit(X_train, a)

BernoulliNB(alpha=0.01, binarize=0.0, class_prior=None, fit_prior=True)

In [147]:
print(classification_report(b,  clf.predict(X_test)))

             precision    recall  f1-score   support

          0       0.89      0.79      0.84    128070
          1       0.72      0.85      0.78     79671

avg / total       0.82      0.81      0.81    207741

